# Liriscat paper experiments
### 1. Init
#### 1.1. Import libraries (necessary)

In [1]:
%load_ext autoreload
%autoreload 2

import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
os.environ["PYTHONHASHSEED"] = "0"
os.environ['CUBLAS_WORKSPACE_CONFIG'] = ":4096:8"

import liriscat
liriscat.utils.set_seed(0)

import logging
import gc
import json
import torch
liriscat.utils.set_seed(0)
import pandas as pd

#### 1.2. Set up the loggers (recommended)

In [2]:
liriscat.utils.setuplogger(verbose = True, log_name="liriscat", debug=True)

### 2. CDM prediction
#### 2.1. Training and testing, sequential version

In [3]:
import warnings

gc.collect()
torch.cuda.empty_cache()

In [4]:
config = liriscat.utils.generate_eval_config(load_params=True, esc = 'error', valid_metric= 'mi_acc', pred_metrics = ["mi_acc"], profile_metrics = ['meta_doa'], save_params=False, n_query=6, num_epochs=100, batch_size=512)
liriscat.utils.set_seed(config["seed"])

config["dataset_name"] = "assist0910"
logging.info(config["dataset_name"])
config['learning_rate'] = 0.0001
config['inner_user_lr'] = 0.0016969685554352153
config['lambda'] = 2.2656270501845414e-06

config['meta_lr'] = 0.5
config['learning_users_emb_lr'] = 0.0005
config['patience'] = 20#5
config['num_inner_users_epochs'] = 3
config['d_in'] = 4
config['num_responses'] = 12

#pred_metrics,df_interp = test(config)

CUDA is available. Using GPU.
[INFO 05:05] math2


In [5]:
def pareto_index2(d) : 
    d_acc = d[0]
    d_meta = d[1]

    r = []

    for i in range(len(d_acc)):
        r.append((0.5-d_acc[i]['mi_acc'])*(0.5-d_meta[i]['meta_doa']))
    return sum(r)

In [6]:
logging.info(f'#### {config["dataset_name"]} ####')
logging.info(f'#### config : {config} ####')
config['embs_path']='../embs/'
config['params_path']='../ckpt/'

gc.collect()
torch.cuda.empty_cache()

# Dataset downloading for doa and rm
warnings.filterwarnings("ignore", message="invalid value encountered in divide")
warnings.filterwarnings("ignore", category=RuntimeWarning)

## Concept map format : {question_id : [category_id1, category_id2, ...]}
concept_map = json.load(open(f'../datasets/2-preprocessed_data/{config["dataset_name"]}_concept_map.json', 'r'))
concept_map = {int(k): [int(x) for x in v] for k, v in concept_map.items()}

## Metadata map format : {"num_user_id": ..., "num_item_id": ..., "num_dimension_id": ...}
metadata = json.load(open(f'../datasets/2-preprocessed_data/{config["dataset_name"]}_metadata.json', 'r'))


## Tensor containing the nb of modalities per question
nb_modalities = torch.load(f'../datasets/2-preprocessed_data/{config["dataset_name"]}_nb_modalities.pkl',weights_only=True)


[INFO 41:03] #### assist0910 ####
[INFO 41:03] #### config : {'seed': 0, 'dataset_name': 'assist0910', 'load_params': True, 'save_params': False, 'embs_path': '../embs/', 'params_path': '../ckpt/', 'early_stopping': True, 'esc': 'error', 'verbose_early_stopping': False, 'disable_tqdm': False, 'valid_metric': 'mi_acc', 'learning_rate': 0.0001, 'batch_size': 512, 'valid_batch_size': 10000, 'num_epochs': 100, 'eval_freq': 1, 'patience': 20, 'device': device(type='cuda'), 'lambda': 2.2656270501845414e-06, 'tensorboard': False, 'flush_freq': True, 'pred_metrics': ['mi_acc'], 'profile_metrics': ['meta_doa'], 'num_responses': 12, 'low_mem': False, 'n_query': 6, 'CDM': 'impact', 'i_fold': 0, 'num_inner_users_epochs': 3, 'num_inner_epochs': 10, 'inner_lr': 0.0001, 'inner_user_lr': 0.0016969685554352153, 'meta_lr': 0.5, 'meta_trainer': 'Adam', 'num_workers': 0, 'pin_memory': False, 'debug': True, 'learning_users_emb_lr': 0.0005, 'd_in': 4} ####


[INFO 38:14] #### config : {'seed': 0, 'dataset_name': 'math2', 'load_params': True, 'save_params': False, 'embs_path': '../embs/', 'params_path': '../ckpt/', 'early_stopping': True, 'esc': 'error', 'verbose_early_stopping': False, 'disable_tqdm': False, 'valid_metric': 'mi_acc', 'learning_rate': 0.0001, 'batch_size': 512, 'valid_batch_size': 10000, 'num_epochs': 100, 'eval_freq': 1, 'patience': 20, 'device': device(type='cuda'), 'lambda': 9.972254466547545e-06, 'tensorboard': False, 'flush_freq': True, 'pred_metrics': ['mi_acc'], 'profile_metrics': ['meta_doa'], 'num_responses': 12, 'low_mem': False, 'n_query': 6, 'CDM': 'impact', 'i_fold': 0, 'num_inner_users_epochs': 3, 'num_inner_epochs': 10, 'inner_lr': 0.0001, 'inner_user_lr': 0.016848380924625605, 'meta_lr': 0.5, 'meta_trainer': 'Adam', 'num_workers': 0, 'pin_memory': False, 'debug': False, 'learning_users_emb_lr': 0.001, 'd_in': 4} ####


In [8]:
meta_trainers = ['GAP']
for meta_trainer in meta_trainers : 
    config['meta_trainer'] = meta_trainer
    logging.info(f'#### meta_trainer : {config["meta_trainer"]} ####')
    for i_fold in range(1,2) : 
        config['i_fold'] = i_fold
            
        logging.info(f'#### i_fold : {i_fold} ####')
        ## Dataframe columns : (user_id, question_id, response, category_id)
        train_df = pd.read_csv(
            f'../datasets/2-preprocessed_data/{config["dataset_name"]}_train_{i_fold}.csv',
            encoding='utf-8', dtype={'student_id': int, 'item_id': int, "correct": float,
                                                                    "dimension_id": int})
        valid_df = pd.read_csv(
            f'../datasets/2-preprocessed_data/{config["dataset_name"]}_valid_{i_fold}.csv',
            encoding='utf-8', dtype={'student_id': int, 'item_id': int, "correct": float,
                                                                    "dimension_id": int})
        test_df = pd.read_csv(
            f'../datasets/2-preprocessed_data/{config["dataset_name"]}_test_{i_fold}.csv',
            encoding='utf-8', dtype={'student_id': int, 'item_id': int, "correct": float,
                                                                    "dimension_id": int})

        train_data = liriscat.dataset.CATDataset(train_df, concept_map, metadata, config,nb_modalities)
        valid_data = liriscat.dataset.EvalDataset(valid_df, concept_map, metadata, config,nb_modalities)
        test_data = liriscat.dataset.EvalDataset(test_df, concept_map, metadata, config,nb_modalities)

        for seed in range(1) :
            config['seed'] = seed
            logging.info(f'#### seed : {seed} ####')

            train_data.reset_rng()
            valid_data.reset_rng()
            test_data.reset_rng()

            S = liriscat.selectionStrategy.Random(metadata,**config)
            S.init_models(train_data, valid_data)
            S.train(train_data, valid_data)
            liriscat.utils.set_seed(0)
            S.reset_rng()
            d = (S.evaluate_test(test_data, train_data, valid_data))
            logging.info(d)
            logging.info(liriscat.utils.pareto_index(d))

    torch.cuda.empty_cache()

[INFO 41:03] #### meta_trainer : GAP ####
[INFO 41:03] #### i_fold : 1 ####
[INFO 41:20] #### seed : 0 ####
[DEBUG 41:20] ------- Abstract model __init__()
[DEBUG 41:20] ----- Meta trainer init : GAP
[INFO 41:20] Random_cont_model
[DEBUG 41:20] ------- Initialize CDM and Selection strategy
[INFO 41:21] compiling CDM model
[INFO 41:25] compiling selection model
[INFO 41:25] ------- START Training
[INFO 41:25] train on cuda


/lustre/fsn1/projects/rech/enh/unv34ei/liriscat/liriscat/selectionStrategy/random.py:30: UserWarning: get_params() Notimplemented
  warnings.warn('get_params() Notimplemented')
  0%|          | 0/100 [00:00<?, ?it/s]

[INFO 05:35] - meta_params: 0.5
[INFO 05:35] - cross_cond: 0.5
[INFO 05:35] - meta_lambda: 0.5
[INFO 05:35] - learning_users_emb: 0.0005
[INFO 05:41] rmse : 0.5449409484863281
[INFO 05:41] valid_rmse : 0.6968554258346558
[INFO 05:41] valid_loss : 0.9390125274658203


  1%|          | 1/100 [00:21<35:47, 21.69s/it]

[INFO 05:55] - meta_params: 0.5
[INFO 05:55] - cross_cond: 0.5
[INFO 05:55] - meta_lambda: 0.5
[INFO 05:55] - learning_users_emb: 0.0005
[INFO 05:59] rmse : 0.554278552532196
[INFO 05:59] valid_rmse : 0.7087960839271545
[INFO 05:59] valid_loss : 0.9394581317901611


...

[INFO 14:37] - meta_params: 0.00390625
[INFO 14:37] - cross_cond: 0.00390625
[INFO 14:37] - meta_lambda: 0.00390625
[INFO 14:37] - learning_users_emb: 3.90625e-06
[INFO 14:42] rmse : 0.5473130345344543
[INFO 14:42] valid_rmse : 0.6998887658119202
[INFO 14:42] valid_loss : 0.9386559128761292


 29%|██▉       | 29/100 [09:22<22:57, 19.40s/it]

[INFO 14:42] -- END Training --
[INFO 14:42] train on cuda



100%|██████████| 6/6 [00:04<00:00,  1.44it/s]


[INFO 14:53] ({0: {'mi_acc': 0.6947683095932007}, 1: {'mi_acc': 0.6942700147628784}, 2: {'mi_acc': 0.6940707564353943}, 3: {'mi_acc': 0.6961634159088135}, 4: {'mi_acc': 0.6958644390106201}, 5: {'mi_acc': 0.6982560753822327}}, {0: {'meta_doa': 0.5289896884487395}, 1: {'meta_doa': 0.5510916448164791}, 2: {'meta_doa': 0.5684174768159787}, 3: {'meta_doa': 0.5906893303588627}, 4: {'meta_doa': 0.6148654919621281}, 5: {'meta_doa': 0.6317061722410084}})
[INFO 14:53] 0.09524922147326688


In [27]:
config['meta_trainer'] = 'GAP'
#config['meta_trainer'] = 'Adam'
for seed in range(1) :
    config['seed'] = seed
    logging.info(f'#### seed : {seed} ####')
    
    train_data.reset_rng()
    valid_data.reset_rng()
    test_data.reset_rng()


    S = liriscat.selectionStrategy.Random(metadata,**config)
    S.init_models(train_data, valid_data)
    S.train(train_data, valid_data)
    liriscat.utils.set_seed(0)
    S.reset_rng()
    d = (S.evaluate_test(test_data, train_data, valid_data))
    print(liriscat.utils.pareto_index(d))
    print(d)
    torch.cuda.empty_cache()

[INFO 04:26] #### seed : 0 ####
[INFO 04:26] Random_cont_model
[INFO 04:26] compiling CDM model
[INFO 04:26] compiling selection model
[INFO 04:26] ------- START Training
[INFO 04:26] train on cuda


/lustre/fsn1/projects/rech/enh/unv34ei/liriscat/liriscat/selectionStrategy/random.py:30: UserWarning: get_params() Notimplemented
  warnings.warn('get_params() Notimplemented')
  0%|          | 0/100 [00:00<?, ?it/s]

[DEBUG 08:24] ------- Epoch : 0
[INFO 08:24] ----- User batch : 0
[DEBUG 08:24] --- Query nb : 0
[DEBUG 08:24] - Update users 
[DEBUG 13:26] --- Query nb : 1
[DEBUG 13:26] - Update users 
